### 피봇테이블과 그룹 분석¶
#### 피봇 테이블(pivot table)¶
- 데이터 열 중에서 두 개를 키(key)로 사용하여 데이터를 선택하는 방법
- Pandas는 피봇 테이블을 만들기 위한 pivot 메서드를 제공
- 첫번째 인수로는 행 인덱스로 사용할 열 이름, 두번째 인수로는 열 인덱스로 사용할 열 이름, 그리고 마지막으로 데이터로 사용할 열 이름을 넣음

In [1]:
data = {
    '도시':['서울']*3+['부산']*3+['인천']*2,
    '연도':['2015', '2010', '2005', '2015', '2010', '2005', '2015', '2010'],
    '인구':[9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 2632035],
    '지역':['수도권']*3+['경상권']*3+['수도권']*2
}
columns=['도시', '연도', '인구', '지역']
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,2632035,수도권


In [2]:
df1.pivot('도시', '연도', '인구')

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,2632035.0,2890451.0


피봇 테이블은 `set_index 명령`과 `unstack` 명령을 사용해서 만들 수도 있다.

In [3]:
df1.set_index(['도시', '연도'])[['인구']].unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN  2632035.0  2890451.0

행 인덱스와 열 인덱스는 하나의 데이터를 찾는 키(key)의 역할을 한다. 즉, 이 값으로 데이터가 유일하게(unique) 결정되어야 한다. 만약 행 인덱스와 열 인덱스 조건을 만족하는 데이터가 2개 이상인 경우에는 에러가 발생한다. 

In [6]:
df1.pivot('지역', '연도', '인구')

ValueError: Index contains duplicate entries, cannot reshape

#### 그룹 분석¶
이렇게 특정 조건에 맞는 데이터가 하나 이상 즉, 그룹을 이루는 경우는 그룹 분석을 해야 한다.
1. 분석하고자 하는 시리즈나 데이터프레임에 groupby 메서드를 호출하여 그룹화를 한다.
2. 그룹 객체에 대해 그룹 연산을 수행한다.

#### groupby 메서드¶
- groupby 메서드는 데이터를 그룹 별로 분류하는 역할을 한다. groupby 메서드의 인수로는 `열 또는 열의 리스트`, `행 인덱스`를 사용한다.
- 연산 결과로 그룹 데이터를 나타내는 GroupBy 클래스 객체를 반환한다. 이 객체에는 그룹별로 연산을 할 수 있는 그룹 연산 메서드가 있다.

#### 그룹 연산 메서드¶
groupby 결과, 즉 GroupBy 클래스 객체의 뒤에 붙일 수 있는 그룹 연산 메서드는 다양하다. 전체 목록을 보려면 다음 웹사이트를 참조한다.

https://pandas.pydata.org/pandas-docs/stable/api.html#groupby

다음은 자주 사용되는 그룹 연산 메서드들이다.

- `size()`, `count()`: 갯수
- `mean()`, `median()`, `min()`, `max()`: 평균, 중앙값, 최소, 최대
- `sum()`, `prod()`, `std()`, `var()`, `quantile()` : 합계, 곱, 표준편차, 분산, 사분위수
- `first()`, `last()`: 가장 첫번째 데이터와 가장 나중 데이터

- `agg()`, `aggregate()`

만약 원하는 그룹 연산이 없는 경우 함수를 만들고 이 함수를 agg()에 전달한다.
또는 여러가지 그룹 연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달한다.
- `describe()`

하나의 그룹 대표값이 아니라 여러개의 값을 데이터프레임으로 구한다.
- `apply()`

describe() 처럼 하나의 대표값이 아닌 데이터프레임을 출력하지만 원하는 그룹 연산이 없는 경우에 사용한다.
- `transform()`

그룹에 대한 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터 자체를 변형한다.

In [11]:
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

,data1,data2,key1,key2
0,1,10,A,one
1,2,20,A,two
2,3,30,B,one
3,4,40,B,two
4,5,50,A,one


In [13]:
# key1 값에 따른 data1, 2의 평균
df2.groupby('key1').sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [20]:
# key1 값에 따른 data1의 평균
df2.data1.groupby(df2.key1).sum(), df2.groupby('key1')['data1'].sum()

(key1
 A    8
 B    7
 Name: data1, dtype: int64, key1
 A    8
 B    7
 Name: data1, dtype: int64)

#### 연습 문제 1¶
key1의 값을 기준으로 data1의 값을 분류하여 합계를 구한 결과를 시리즈가 아닌 데이터프레임으로 구한다.

In [22]:
df2.groupby('key1').sum()[['data1']]

,data1
key1,
A,8
B,7


복합 키 (key1, key2) 값에 따른 data1의 합계는 리스트를 사용한다.

In [25]:
df2.data1.groupby([df2.key1, df2.key2]).sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

이 결과를 `unstack`명령으로 피봇테이블 형태로 만들어 가독성을 높일 수 있다.

In [29]:
df2.groupby([df2['key1'], df2['key2']]).sum()['data1'].unstack('key2')

key2,one,two
key1,,
A,6,2
B,3,4


In [40]:
df1[['인구']].groupby([df1['지역'], df1['연도']]).agg(sum)

인구
지역  연도            
경상권 2005   3512547
    2010   3393191
    2015   3448737
수도권 2005   9762546
    2010  12263517
    2015  12794763

In [36]:
df1[['인구']].groupby([df1['지역'], df1['연도']]).sum().unstack('연도')

인구                    
연도      2005      2010      2015
지역                              
경상권  3512547   3393191   3448737
수도권  9762546  12263517  12794763

150송이의 붓꽃에 대해 측정한 데이터에서 각 붓꽃 종별로 가장 큰 값과 가장 작은 값의 비율을 구해보자

In [38]:
import seaborn as sns
iris = sns.load_dataset("iris")

In [39]:
iris.tail()

,sepal_length,sepal_width,petal_length,petal_width,species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


In [42]:
def ratio_max_min(x):
    return x.max() / x.min()
iris.groupby('species').agg(ratio_max_min)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


`describe` 메서드를 사용하면 다양한 기술 통계(descriptive statistics)값을 한 번에 구한다. 그룹별로 하나의 스칼라 값이 아니라 하나의 데이터프레임이 생성된다는 점에 주의